# Region and Artifact Detection Tutorial

This tutorial walks you to perform patch based region and artifact detection using KRONOS. The workflow involves extracting token-specific embeddings from input images and classifying each token.

## Prerequisites

To follow this tutorial, ensure you have the following data prepared:

 - A set of annotated 256x256 patches extracted from multiplex images and saved as .h5 format under patches folder in your project directory. 
 - **patch_annotations.csv** cantaining two columns `patch_name` and `label`.
 - **marker_info_with_metadata.csv**: A CSV file containing:
    - `channel_id`: Identifier for the image channel.
    - `marker_name`: Name of the marker (e.g., DAPI, CD20).
    - `marker_id`: Unique ID for the marker.
    - `marker_mean`: Mean intensity value for normalization.
    - `marker_std`: Standard deviation for normalization.

 **Note**: Refer to the **[1-Data-Download-And-Preprocessing](https://github.com/mahmoodlab/KRONOS/blob/main/tutorials/1%20-%20Data-Download-And-Preprocessing.ipynb)** tutorial to prepare data and refer to **[3 - Patch-phenotyping](https://github.com/mahmoodlab/KRONOS/blob/main/tutorials/3%20-%20Patch-phenotyping.ipynb)** tutorial to exptract patches from multiplex images.

## Step 1: Experiment Configuration

In this section, we define the configuration and hyperparameters required for the workflow. Ensure your dataset in the project directory is organized as follows:

### Dataset Structure
- **`dataset/multiplex_images/`**: Contains the multiplex image files (e.g., `.tiff`).
- **`dataset/marker_info_with_metadata.csv`**: A CSV file containing marker metadata.
- **`dataset/patch_annotations.csv`**: A CSV file containing ground truth annotations.

### Output Directories
The following directories in the project directory will be generated during the workflow:
- **`patches/`**: Stores the extracted cell-centered patches in `.h5` format.
- **`features/`**: Stores the extracted features in `.npy` format.
- **`folds/`**: Contains cross-validation folds for training, validation, and testing.
- **`results/`**: Stores the results for each fold and aggregated metrics.

In [ ]:
from utils import RegionArtifactClassification 

# Define the root directory for the project
project_dir = "path/to/project/dir/"  # Replace with your actual root directory

# Configuration dictionary containing all parameters for the pipeline
config = {
    # Dataset-related parameters, feel free to modify these paths according to your dataset structure
    "image_dir_path": f"{project_dir}/dataset/multiplex_images/",  # Path to the multiplex image file
    "cell_mask_dir_path": f"{project_dir}/dataset/cell_masks/",  # Path to the cell mask file
    "marker_info_with_metadata_csv_path": f"{project_dir}/dataset/marker_info_with_metadata.csv",  # Path to the marker metadata CSV file
    "gt_csv_path": f"{project_dir}/dataset/cell_annotations.csv",  # Path to the ground truth annotations CSV file
    
    # Output directories for intermediate and final results
    "patch_dir": f"{project_dir}/patches/",  # Directory to save extracted patches
    "feature_dir": f"{project_dir}/features/",  # Directory to save extracted features
    "fold_dir": f"{project_dir}/folds/",  # Directory to save cross-validation folds
    "result_dir": f"{project_dir}/results/",  # Directory to save final results
    
    # Model-related parameters
    "checkpoint_path": "hf_hub:MahmoodLab/kronos",  # Path to the pre-trained model checkpoint (local or Hugging Face Hub)
    "hf_auth_token": None,  # Authentication token for Hugging Face Hub (if checkpoint is from the Hugging Face Hub)
    "cache_dir": f"{project_dir}/models/",  # Directory to cache KRONOS model if downloading model from Hugging Face Hub
    "model_type": "vits16",  # Type of pre-trained model to use (e.g., vits16)
    "token_overlap": True,  # Whether to use token overlap during feature extraction

    # Patch extraction parameters
    "patch_size": 32,  # Size of the patches to extract (e.g., 64x64 pixels)
    "patch_stride": 16,  # Stride for patch extraction (e.g., 32 pixels)
    "token_size": 16,  # Size of the tokens for the model (e.g., 16x16 pixels)
    "token_stride": 8,  # Stride for token extraction (e.g., 16 or 8 pixels)

    # Feature extraction parameters
    "marker_list": ['DAPI'],  # List of markers to be used for patch phenotyping
    "marker_max_values": 65535.0,  # Maximum possible value marker image, depends on image type (e.g., 255 for uint8, 65535 for uint16)
    "patch_batch_size": 16,  # Batch size for patch-based data loading
    "num_workers": 4,  # Number of workers for data loading

    # Logistic regression parameters
    "feature_batch_size": 256,  # Batch size for feature-based data loading
    "n_trials": 25,  # Number of trials for Optuna hyperparameter optimization
    "C_low": 1e-10,  # Lower bound for the regularization parameter (C) in logistic regression
    "C_high": 1e5,  # Upper bound for the regularization parameter (C) in logistic regression
    "max_iter": 10000,  # Maximum number of iterations for logistic regression training
}
obj = RegionArtifactClassification(config)

## Step 2: Dummy Dataset Generation (Optional) 

If you are not sure about exact input format of the data then run following script to generate .h5 patches that cantain random values but will be useful to run this tutorial end-to-end.

In [ ]:
obj.generate_dummy_data()

## Step 3: Feature Extraction

Extract features from patches using a pre-trained model. This step processes patches of multiplex images, applies a pre-trained model to extract meaningful feature vectors, and saves them for downstream analysis.

In [ ]:
obj.feature_extraction()

## Step 4: Generate Data Folds

Generate data folds for training, validation, and testing. This tutorial split the patches listed in the annotation.csv into four equal sets based on their order in the list. Each set represents one fold.

In [ ]:
obj.folds_generation()

## Step 5: Train Region or Artifact Detection Model

Train a logistic regression model on the training data, validate it on the validation data, and evaluate it on the test data. Results for each fold are saved in the output directory.

In [ ]:
obj.train_classification_model()

## Step 6: Evaluate Model

Read the test results for each fold, compute evaluation metrics (F1-Score, Balanced Accuracy, Average Precision, and ROC AUC), and calculate the average and standard deviation across all folds.

In [ ]:
obj.eval_classification_model()
obj.calculate_results()